The data were used is available in [Database source](https://github.com/owid/covid-19-data/tree/master/public/data)

#### Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
import math
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
import scipy.stats as stats
import scipy.linalg as linalg

#### Loading Data

Loading data fill NA with 0 and get all contries

In [4]:
world = pd.read_csv('world.csv',sep=',')
world.fillna(0,inplace=True)
countries = world['location'].unique()

Choose the number of days to create a gaussian model


In [5]:
begin = 1
end   = 150

Variable for configuration

In [6]:
graphicDays = 200
factor = 40
significance = 0.05
directory = 'calculated/'

[Referência para o p-value a partir do F](https://www.programcreek.com/python/example/113313/scipy.stats.f.cdf)

### Returns the p-value using the computed F-statistic

#### Returns

p : float
    The computed p-value given the found F-statistic and the group and residual degrees of freedom.

#### Notes

The code `cdf` method from Scipy's code `stats.f` class is used to find the p-value.

#### References

Rencher, A. (n.d.). Methods of Multivariate Analysis (2nd ed.). Brigham Young University: John Wiley & Sons, Inc.

#### Códigos
`
p = 1 - f.cdf(self.f_statistic,
      self.group_degrees_of_freedom,
      self.residual_degrees_of_freedom)
`

#### Used model

In [7]:
def model_fun(x,a,b,c):
    return  a * np.exp(-(x - b) ** 2 / (2 * c ** 2))


#### Chow's test

In [8]:
def chow_test(y1,y2):
    x1 = ar(list(range(len(y1))))
    x2 = ar(list(range(len(y2))))
    x3 = ar(list(range(len(y1)+len(y2))))
    y3 = np.append(y1,y2)
    
    x1 = sm.add_constant(x1)
    x2 = sm.add_constant(x2)
    x3 = sm.add_constant(x3)
    
    model1 = sm.OLS(y1,x1).fit()
    model2 = sm.OLS(y2,x2).fit()
    model3 = sm.OLS(y3,x3).fit()
    
    RSSall = model3.ssr
    RSSbefore = model1.ssr
    RSSafter = model2.ssr
    
    N1 = len(x1)
    N2 = len(x2)
    chow = ((RSSall-(RSSbefore+RSSafter))/2.0)/((RSSbefore+RSSafter)/(N1+N2-4.0))
    
    df1 = 2.0
    df2 = len(x1) + len(x2) - 4.0
 
    p_val = 1 - stats.f.cdf(chow, df1, df2)
    return p_val

In [21]:
significant = 0
nonSignificant = 0
out = 0
for country in countries:
    significantDate = None
    dataCountry = world[world['location'] == country]
    density = dataCountry['population_density'].values[0]
    if(density <= 0):
        density = 25

    means = dataCountry['new_cases'].rolling(7).mean()
    means[:6] = dataCountry['new_cases'].values[:6]
    dateX = dataCountry['date']
    x = ar(range(len(means)))
    y = ar(means.values)
    

    coefs = []
    dates = []
    endding = end
    if(len(y) < endding):
        endding = len(y)
    getIn = False
    for f in range(begin,endding):
        if((y[f] > 2*density and y[f-1] > 2*density) or f >= 30):
            trainX = x[:f]
            trainY = y[:f]
            if(trainY.max() > 3):
                try:
                    popt,pcov = curve_fit(model_fun,trainX,trainY,[1, 1, 100], maxfev=5000)
                except:
                    try:
                        popt,pcov = curve_fit(model_fun,trainX,trainY,[10, 1, 1], maxfev=5000)
                    except:
                        continue
            else:
                continue
            dates.append(dateX.values[f])
            coefs.append(popt[0])
            
            if(significantDate == None and popt[0] >= factor*density and len(coefs) > 3 and f >= 14):
                significantDate = dateX.values[f]
                getIn = True
                before = y[f-14:f]
                after = y[f:f+14]

                df = pd.DataFrame()
                g1 = np.full(len(before),1)
                g2 = np.full(len(after),2)
                
                d1 = list(zip(g1,before))
                d2 = list(zip(g2, after))
                
                df = df.append(d1)
                df = df.append(d2)
                df.reset_index(drop=True, inplace=True)

                df.columns = ['Group', 'Cases']
                results = smf.ols('Cases ~ C(Group)', data=df).fit()

                pchow = chow_test(before,after)
                if(results.pvalues[1] <= significance and pchow <= significance):
                    significant = significant + 1
                else:
                    nonSignificant = nonSignificant + 1

    if(not getIn and len(coefs) > 0):
        out = out + 1
        
    if(significantDate != None):
        print(country)
        dates = ar(dates)
        fig = plt.figure(figsize=(16,9))
        plt.plot(dates,coefs,'b+:',label='Gaussiana')
        plt.yscale('log')
        plt.title(country)
        idx = list(range(0,len(dates),2))
        plt.xticks(dates[idx], dates[idx], rotation='vertical')
        plt.hlines(factor*density,0,len(coefs))
        plt.legend()

        try:
            plt.savefig(directory+country+'_gaussian.png')
        except:
            print('ERRO:',country)
            pass
        fig.clf()
        plt.close()
    
        fig = plt.figure(figsize=(16,9))
        plt.plot(dateX[:graphicDays],y[:graphicDays],label='New Cases')

        plt.vlines(significantDate,means[:graphicDays].min(),means[:graphicDays].max())
        plt.title(country)
        idx = list(range(0,len(dateX[:graphicDays]),10))
        dateX = dateX.reset_index(drop=True)
        plt.xticks(dateX[idx], dateX[idx], rotation='vertical')
        plt.legend()

        try:
            plt.savefig(directory+country+'_curva.png')
        except:
            print('ERRO:',country)
            pass

        fig.clf()
        plt.close()
    #break
    
    
print('END', significant, nonSignificant, out, significant+nonSignificant, significant+nonSignificant+out, len(countries)-significant-nonSignificant-out, len(countries))

/home/raphael/.local/lib/python3.6/site-packages/scipy/optimize/minpack.py:787: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Afghanistan 3.4467764438101263e-10
Africa 6.593652577808076e-10
Albania 5.869709006343797e-07
Algeria 5.181167722702819e-06
Andorra fora
Angola 0.0001557813624260258
Antigua and Barbuda 0.0007516421120277423
Argentina 7.725660463092637e-09
Armenia 2.7061698891566688e-08
Asia 1.2384507307578361e-09
Australia 6.646083749602794e-05
Austria fora
Azerbaijan 1.4463192508568895e-05
Bahamas fora
Bahrain 8.194455522433007e-05
Bangladesh fora
Barbados fora
Belarus 8.285897956459051e-12
Belgium 1.1583874822269552e-06
Belize fora
Benin 2.3099599147610657e-10
Bolivia 1.2925555800900236e-07
Bosnia and Herzegovina 3.925454948169911e-09
Botswana 0.28999723824619755
Brazil 7.24878343073568e-11
Brunei fora
Bulgaria 5.310359070408066e-05
Burkina Faso fora
Burundi 0.526690736482871
Cambodia fora
Cameroon 6.387934202427651e-07
Canada 0.000505858707614583
Cape Verde 2.021047647265454e-05
Central African Republic 4.01251998509658e-07
Chad fora
Chile 1.1303409175848528e-08
China 1.2131635873850863e-09
Colombi

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/home/raphael/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/raphael/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/raphael/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


Grenada nan
Guatemala 1.1225406668639962e-09
Guinea fora
Guinea-Bissau fora
Guyana 0.00011056736436210619
Haiti 0.0014819075723233866
Honduras 0.011509380624530646
Hungary fora
Iceland 2.8050534028008257e-08
India 2.0872626007223607e-05
Indonesia 1.848457560546312e-05
International fora
Iran 1.8752947905269558e-08
Iraq 1.3917808661651145e-05
Ireland 1.764586180709739e-05
Israel 2.7813452467245294e-11
Italy 1.2702530587267272e-07
Jamaica fora
Japan 1.548807526478338e-08
Jordan 0.09711711638074558
Kazakhstan 3.975581505590302e-09
Kenya 2.905430778103939e-07
Kosovo fora
Kuwait 1.5123550568525951e-06
Kyrgyzstan 0.38157864482484805
Latvia fora
Lebanon 0.2850499129854625
Lesotho fora
Liberia 0.047084651250427115
Libya 8.625899708369656e-10
Liechtenstein fora
Lithuania fora
Luxembourg fora
Madagascar 0.00013946662486873408
Malawi 0.01470760285593435
Malaysia fora
Maldives 1.5657296883538816e-05
Mali fora
Malta fora
Mauritania 1.8267141560344822e-08
Mauritius fora
Mexico 4.6170039998398453e-10